# Pandas = **PAN**el **DA**ta**S**ets


>  *pandas* provides high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

* General purpose data munger and ```numpy``` array wrapper  
* Persist to / read from variety of data sources including Excel 
* Two core data structures: a ```Series``` for 1d data and a ```DataFrame``` for 2d data
* ```DataFrames``` are indexed by rows and columns and all operations are index-aware
* Joins/merge
* Summarize, transform
* melt, stack/unstack, pivot tables
* Excellent time series support 
* Good integration with Jupyter for viewing data 
* Graceful handling of missing values 
* Nice integration with Python string handling 
* Plotting

See [10 minute intro to pandas](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

## Functions We Will Discuss

* DataFrame
* head, tail, describe
* unique, value_counts
* read_csv
* loc, slices, xs
* create_index, reset_index 
* MultiIndex 
* query 
* pivot, stack and unstack
* **concat**, append, keys 
* pivot_table (crosstab), pivot 
* **merge** (indicator) and join
* groupby (.groups, .get_group, as_index)
* sum, mean, std etc. 
* aggregate
* transform (same size as input whiten)
* apply
* assign 
* plot

## Functions not covered but check out on your own
* map (series), applymap (dataframes) 
* from_dict
* rename 
* melt
* evaluate 
* str
* dt
* style

# Seaborn Plotting 

```pandas``` + ```seaborn``` $\approx$ ```tibbles``` + ```ggplot```

* relplot  = relational plots, line, scatter 
* catplot = scatter plot with categorical, box, swarm, bar, count 
* jointplot, pairplot, distplot, kdeplot
* lmplot, regplot, residplot 
* heatmap, clustermap 
* faceting, row/column plots

In [ ]:
# the basics 
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
import re

# nice printing of dataframes 
from IPython.display import display, HTML

# other setup 
warnings.filterwarnings("ignore")
%matplotlib inline
%load_ext line_profiler
np.set_printoptions(linewidth =  160)

# handy utility 
import textwrap 
def wdid(ob, ex=False):
    ''' what does object do? 
    '''
    print('\n'.join(textwrap.wrap(' '.join([i for i in dir(ob) if i[0] != '_']), 80)))
    if ex:
    # optional pause for something more advanced... 
        for m in [ i for i in dir(np) if i[0] >= 'a' and i[0]<='z']:
            print(f'\n\n{m}\n{"="*len(m)}\n')
            print(np.__getattribute__(m).__doc__)

# Basic Pandas DataFrame Creation and Manipulation 

## Create from ```numpy``` array 

In [ ]:
x = np.random.randn(5,5); x

In [ ]:
df0 = pd.DataFrame(x)
df0

In [ ]:
df0.columns = list('abcde')
df0

In [ ]:
# acess a column like a dictionary (as elements)
df0['b']

In [ ]:
# or as attributes if name does not have spaces nor clashes with functions 
df0.b

In [ ]:
# create new columns 
# strings in python are unicode 
df0['id'] = list('αβγδϵ')
df0['region'] = ['East', 'North', 'South', 'South', 'West']
df0

In [ ]:
# frills
df1 = df0.set_index('id')
df1.columns.name = 'quantity'
df1

## Create from dictionary 

Entries in dictionary correspond to columns by default. Somewhat analogous to ```mutate```. See also ```assign```.

In [ ]:
df2 = pd.DataFrame({'region': ['East']*3 + ['South', 'West'], 
                   'type': ['Urban', 'Suburban', 'Rural', 'Rural', 'Urban'], 
                   'a': np.random.randn(5) * 100, 
                   'b': np.arange(10,15),
                   'c': np.arange(5, dtype=np.float)}, 
                  index=pd.Index(list('βγδϵω'), name='id'))
df2

## Combine 

In [ ]:
df3 = pd.concat( (df1, df2), sort=True) 
df3

In [ ]:
df3v = pd.concat( (df1, df2), sort=True, axis=1) 
df3v

In [ ]:
df1.to_excel('/temp/foo.xlsx', sheet_name='Test1')

In [ ]:
pd.read_excel("/temp/foo.xlsx", 'Test1', index_col=0, na_values=['NA'])

In [ ]:
df1 / df2

In [ ]:
df1.dtypes

In [ ]:
display(df1.select_dtypes(np.number))
df2.select_dtypes(np.number)

In [ ]:
df1.select_dtypes(np.number) / df2.select_dtypes(np.number)

In [ ]:
df1['sina'] = np.sin(df1.a)
df1

In [ ]:
df1['$\int_0^2 e^{-a^2/2}da$'] = 12

In [ ]:
df1

In [ ]:
del df1['$\int_0^2 e^{-a^2/2}da$']
df1

In [ ]:
df1.corr()

In [ ]:
df1.append(df1).sort_index()

In [ ]:
df3 = pd.concat((df1,df2), sort=True, keys=['df1', 'df2'], names=['src'])
df3

In [ ]:
df3.region.unique()

In [ ]:
df3.region.value_counts()

In [ ]:
df2.index

In [ ]:
df3.index

In [ ]:
df3.index.get_level_values(1).unique()

In [ ]:
df3.columns

In [ ]:
df3.loc['df1']

In [ ]:
df3.loc[:, 'a']

In [ ]:
df3.loc['a']

In [ ]:
df3.a

In [ ]:
df3[['a', 'b']]

In [ ]:
df3.loc[:, 'a':'c']

In [ ]:
df3.unstack()

In [ ]:
df3.unstack(0)

In [ ]:
df3.a < 0 

In [ ]:
df3[df3.a < 0]

In [ ]:
df3.query( ' a < 0 ')

In [ ]:
df3.loc['df1']

In [ ]:
df3.loc['β']

In [ ]:
df3.loc[(slice(None), 'β'), :]

In [ ]:
df3.loc[(slice(None), slice('β','δ')), :]

In [ ]:
df3 = df3.sort_index()

In [ ]:
df3.loc[(slice(None), 'b')], df3.loc[:, 'b']

In [ ]:
df3.xs('β', level=1)

In [ ]:
df3.xs('b', axis=1)

In [ ]:
# integer valued loc using row and columns, zero based!
df3.iloc[0:2, 1:]

In [ ]:
df4 = df3.reset_index()  # can also reset just one level; can optionally drop, i.e. not create column 
df4

In [ ]:
df4['idx'] = df4.src + df4.id

In [ ]:
df4

In [ ]:
df4.pivot(index='idx', columns='region', values='a')

In [ ]:
df4.pivot(index=['src', 'id'], columns='region', values=['a', 'sina'])

In [ ]:
df4.pivot_table(index=['src', 'id'], columns='region', values=['a', 'sina'])

In [ ]:
g4 = df4.groupby(by='src') 
# to group by index use level=

In [ ]:
g4.groups 

In [ ]:
g4.get_group('df1')

In [ ]:
df4.loc[g4.groups['df1']]

In [ ]:
g4.sum()

In [ ]:
g4.aggregate(sum)

In [ ]:
g4.agg([sum, np.std, np.min, np.max, np.size])

In [ ]:
g4.agg({'a' : [sum, np.std, np.min, np.max, np.size], 'b': [sum, np.std] })

In [ ]:
agg_funs =  [sum, np.std, np.min, np.max, np.size]
g4.agg({'a' : agg_funs, 'b': agg_funs[:2] })

In [ ]:
g4.apply(lambda x : display(x))
# notice first group printed twice 

In [ ]:
x = g4.apply(lambda x : (x.a * x.b).cumsum())
x

In [ ]:
type(x)

In [ ]:
df4.loc[:, 'calc'] = x.reset_index(drop=True)  # or x.values, the issue is with the index 
df4

In [ ]:
y = g4.get_group('df2')
y

In [ ]:
pd.Series( (y.a * y.b).values, name='ab', index=range(len(y)))

In [ ]:
g4.apply( lambda y : (y.a * y.b))

In [ ]:
g4.apply( lambda y : pd.Series((y.a * y.b).values))
# note the index drives the difference - here we have removed the index 

In [ ]:
g4.apply( lambda y : pd.DataFrame({'a': y.a.values, 'b': y.b.values,  'ab': (y.a * y.b).values, 
                                   'sb': y.b.cumsum(), 'db' : y.b.diff() }) )  

# Manipulating Triangle Data in Pandas 1: WC Triangles  


### Load Data and Basic Cleansing

In [ ]:
cas = pd.read_csv(r'http://www.casact.org/research/reserve_data/wkcomp_pos.csv')

In [ ]:
cas.head()  # cas.tail(), cas.head(20)

In [ ]:
cas.describe()

# Exerciess

1. List the unique companies in the database
1. Add a column for loss ratio (IncurLoss_D to EarnedPremNet_D) 
1. Create a table rows = companies, columns = loss ratio for the ultimate (DevelopmentLag==10) evaluation. Remember ?pd.pivot to get help on a function 
1. Create a dataframe showing company and total premium and incurred loss (IncurLoss_D to EarnedPremNet_D) over all 10 years for the most recent evaluation. Add loss ratio column 

In [ ]:
cas.GRNAME.unique()

In [ ]:
cas['LR'] = cas.IncurLoss_D / cas.EarnedPremNet_D
cas.query('DevelopmentLag == 10').pivot(index='GRNAME', columns='AccidentYear', values='LR').head(10)

In [ ]:
cas.query('DevelopmentLag == 10').replace([-np.inf, np.inf], np.nan).pivot(columns='GRNAME', index='AccidentYear', values='LR').plot(kind='line', legend=False, ylim=[0,3], alpha=0.1, color='blue')
cas.query('DevelopmentLag == 10').replace([-np.inf, np.inf], np.nan).groupby('AccidentYear')['LR'].mean().plot(color='red')

In [ ]:
r = cas.query(' AccidentYear + DevelopmentLag == 1998 ').groupby('AccidentYear').agg({'IncurLoss_D' : sum, 'EarnedPremNet_D' : sum})
r['LR'] = r.iloc[:, 0] / r.iloc[:, 1]
r

In [ ]:
# make the pandas dataframe look more triangle like
triangle_frame = cas.query(' DevelopmentYear <= 1997 ').pivot_table(values='CumPaidLoss_D', 
                                                                    index=['GRNAME','AccidentYear'],
                                                                    columns='DevelopmentLag')

In [ ]:
triangle_frame.head(20)

In [ ]:
triangle_frame.tail(10)

In [ ]:
# get rid of zero triangles using filter 
triangle_frame = triangle_frame.groupby(level=0).filter(lambda x : np.nansum(x)  > 0)
triangle_frame.iloc[10:20, :]

In [ ]:
# can filter out smaller triangles too...based on total loss volume of triangle 
triangle_frame.groupby(level=0).filter(lambda x : x.sum().sum()  > 100000).shape

In [ ]:
triangle_frame.iloc[-10:, :]

### Age-to-age factors

In [ ]:
triangle_frame.iloc[-10:, 1:] / triangle_frame.iloc[-10:, :-1]

In [ ]:
triangle_frame.iloc[-10:, 1:].values / triangle_frame.iloc[-10:, :-1].values

In [ ]:
# best of both worlds 
triangle_frame.iloc[-10:, 1:].values / triangle_frame.iloc[-10:, :-1]

In [ ]:
ata_df = triangle_frame.iloc[:, 1:].values / triangle_frame.iloc[:, :-1]
ata_df.tail(10)

### LDFs and CDFs

In [ ]:
triangle_frame.head()

In [ ]:
ata_df.head()

In [ ]:
ldf_df = ata_df.groupby(level=0).mean().fillna(1.)

In [ ]:
ldf_df.head()

In [ ]:
# cdfs need cumulative product in reverse...easy to reverse and re-reverse, axis=1 for columns 
cdf_df = ldf_df.iloc[:, ::-1].cumprod(axis=1).iloc[:, ::-1]
cdf_df[10] = 1.
cdf_df.head()

### Ultimates and IBNR

In [ ]:
# pulling off the diagonal is a bit tricky 
diag_df = triangle_frame.groupby(level=0).apply(lambda x : pd.Series(np.diagonal(x.values[:, ::-1])[::-1], index=range(1,11)))
diag_df.head(10)

In [ ]:
# check 
triangle_frame.iloc[10:20, :]

In [ ]:
ult_df = (diag_df * cdf_df).fillna(0)
ibnr_df = ult_df - diag_df
ult_df.head(10)

In [ ]:
ibnr_df.head()

### The business questions answered by our model

In [ ]:
# ibnr by year 
ibnr_df.sum(axis=0)

In [ ]:
ibnr_df.columns = pd.RangeIndex(1988, 1998)

In [ ]:
ibnr_df.replace(np.inf, np.nan).sum(axis=0) 

In [ ]:
# top 10 cos for ibnr
ibnr_df.replace(np.inf, np.nan).sum(axis=1).sort_values(ascending=False)[:10]

In [ ]:
# alternatively  
ibnr_df.replace(np.inf, np.nan).sum(axis=1).nlargest(10)

In [ ]:
# highest 12:ult cdf
cdf_df.loc[cdf_df.loc[:, 1] < np.inf, 1].nlargest(10)

In [ ]:
# lowest 12:24 ldf
ldf_df.loc[:, 2].nsmallest(10)

In [ ]:
# 95% confidence intervals for each cdf
cdf_df.replace(np.inf, np.nan).describe([0.025, 0.975]).loc[['count', '2.5%', 'mean', '97.5%'], :]

## Numpy Compared to Pandas: Code and Performance

In [ ]:
# start from clean slate...this code will be common 
triangle_frame = cas.query(' DevelopmentYear <= 1997 ').pivot_table(values='CumPaidLoss_D', 
                                                                    index=['GRNAME','AccidentYear'],
                                                                    columns='DevelopmentLag')

###  numpy version

In [ ]:
def develop_np(triangle_frame):
    '''
    create latest ldfs, cdfs, diagonal, ultimate and ibnr ndarrays from
    input pandas dataframe:
    
        pd.pivot_table(cas[cas['DevelopmentYear']<=1997], 
                                values='CumPaidLoss_D', 
                                index=['GRNAME','AccidentYear'], 
                                columns='DevelopmentLag')

    John's code
    
    '''
    
    # use reshape method to create a 3-D Matrix of triangles
    # triangle array is a set 10x10 triangles for more than 100 companies.
    triangle_array = np.array(triangle_frame).reshape(
        len(cas['GRNAME'].unique()),
        len(cas['AccidentYear'].unique()),
        len(cas['DevelopmentLag'].unique())
    )
    
    # get rid of completely empty triangles
    triangle_sum = np.nansum(np.nansum(triangle_array, axis=1),axis=1)
    triangle_array = triangle_array[triangle_sum!=0,:,:]
    triangle_array[triangle_array==0]=np.nan

    # use slicing to create age-to-age factors
    ata_array = triangle_array[:,:-1,1:]/triangle_array[:,:-1,:-1]
    
    # create an array of LDFs, by taking simple averages of the age-to-age factors; default missing to 1
    ldf_array = np.nanmean(ata_array, axis=1)
    ldf_array[np.isnan(ldf_array)] = 1.0

    # create an array of CDFs with a tail factor from our LDFs
    cdf_array = ldf_array[:,::-1].cumprod(axis=1)[:,::-1]
    tail_factor = 1.0
    cdf_array = np.append(cdf_array,np.expand_dims(np.repeat(tail_factor,cdf_array.shape[0]),1),axis=1)[:,::-1]

    # strip latest diagonal and develop 
    latest_diagonal = np.nan_to_num(np.diagonal(triangle_array[:,::-1,],axis1=1,axis2=2)[:, ::-1])
    ultimate = latest_diagonal * cdf_array
    ibnr = ultimate - latest_diagonal
    
    # return the interesting bits 
    return triangle_array, triangle_sum, ldf_array, cdf_array, latest_diagonal, ultimate, ibnr

In [ ]:
%%timeit
# %%prun -s "time" -l 20
triangle_array, triangle_sum, ldf_array, cdf_array, latest_diagonal, ultimate, ibnr = develop_np(triangle_frame)

In [ ]:
triangle_array, triangle_sum, ldf_array, cdf_array, latest_diagonal, ultimate, ibnr = develop_np(triangle_frame)

### pandas Version 

In [ ]:
def develop_pd(triangle_frame):
    '''
    Same thing in pandas
    '''

    triangle_frame1 = triangle_frame.groupby(level=0).filter(lambda x : np.nansum(x)  > 0)

    # ata factors, picks up index from second data frame 
    ata_df = triangle_frame1.iloc[:, 1:].values / triangle_frame1.iloc[:, :-1] 

    # ldfs with default 1 and tail factor in column 10
    ldf_df = ata_df.groupby(level=0).mean().fillna(1.)
    ldf_df[10] = 1.0

    # cdfs 
    cdf_df = ldf_df.iloc[:, ::-1].cumprod(axis=1).iloc[:, ::-1]

    # diagonal
    diag_df = triangle_frame1.groupby(level=0).apply(lambda x : pd.Series(np.diagonal(x.values[:, ::-1])[::-1], index=x.columns))

    # ultimate and ibnr
    ult_df = (diag_df * cdf_df).fillna(0)
    ibnr_df = ult_df - diag_df
    ibnr_df['Tot'] = ibnr_df.sum(1)

    # return interesting bits 
    return ldf_df, cdf_df, diag_df, ult_df, ibnr_df

In [ ]:
%%timeit
# %%prun -s "time" -l 20
ldf_df, cdf_df, diag_df, ult_df, ibnr_df = develop_pd(triangle_frame)

In [ ]:
ldf_df, cdf_df, diag_df, ult_df, ibnr_df = develop_pd(triangle_frame)

Check we get the same answer 

Look at the pieces

In [ ]:
display(ibnr_df.head(20).style)
display(pd.DataFrame(ibnr[:,::-1]).head(20).style)

In [ ]:
display(ult_df.head(10))
display(pd.DataFrame(ultimate).iloc[0:10, ::-1])

In [ ]:
display(diag_df.head(10))
display(pd.DataFrame(latest_diagonal).iloc[:10, 10::-1])

In [ ]:
display(cdf_df.head(10))
display(pd.DataFrame(cdf_array).iloc[0:10, ::-1])

In [ ]:
display(ldf_df.head(10))
display(pd.DataFrame(ldf_array).head(10))

# More Fun with More Triangles... 

Load and develop all triangles in the CAS database.



In [ ]:
N1 = pd.read_csv(r'http://www.mynl.com/RPM/masterdata.csv')

In [ ]:
N1.head()

In [ ]:
N1.describe().style

In [ ]:
N1.Line.unique()

In [ ]:
bit = N1.query(' Lag == 10 ')[['GRName', 'Line', 'UltIncLoss', 'EarnedPrem']]  # .head(1000).copy()

In [ ]:
bit.groupby('GRName').agg({ 'EarnedPrem': sum } ).sort_values('EarnedPrem', ascending=False).head(20) 

In [ ]:
N2 = N1.set_index(keys=['GRName', 'Line', 'AY', 'Lag'])

In [ ]:
N2.head(10)

In [ ]:
big_cos = list( N1.query(' Lag == 10 ').groupby('GRName')[['EarnedPrem']].sum().nlargest(20, 'EarnedPrem').index ) 
big_cos

In [ ]:
bit = N1.query(' AY + Lag <= 1998 ')[['GRName', 'Line', 'PaidLoss', 'CaseIncLoss', 'UltIncLoss', 'EarnedPrem', 'AY', 'Lag']] 
# just the big cos using isin
bit = bit.loc[bit.GRName.isin(big_cos), :]
bit.head(20)

In [ ]:
# a particular randomly selected company 
sfm = 'State Farm Mut Grp' 

In [ ]:
G = pd.pivot_table(N1.query(' AY + Lag <= 1998 and GRName == @sfm '), values=['CaseIncLoss', 'PaidLoss'], index=['GRName', 'Line', 'AY'], columns='Lag')
G.iloc[20:30, :]

In [ ]:
def add_link_ratios_from_raw_data(N1, opt_filter=''):
    '''
    Add link ratios to loss triangles
    e.g. opt_filter = " and GRName=='State Farm Mut Grp' "
    '''
    G = pd.pivot_table(
            N1.query(" AY+Lag <= 1998 " + opt_filter ), 
            values=['PaidLoss', 'CaseIncLoss'], 
            index=['GRName', 'Line', 'AY'], 
            columns='Lag'
        )

    return pd.concat((G, 
                      pd.DataFrame(G.iloc[:, 1:10].values / G.iloc[:, 0:9].values, 
                                   index=G.index, 
                                   columns=pd.MultiIndex.from_tuples([('CaseIncLink', i) for i in range(1,10)])),
                      pd.DataFrame(G.iloc[:, 11:].values / G.iloc[:, 10:-1].values, 
                                   index=G.index, 
                                   columns=pd.MultiIndex.from_tuples([('PaidLink', i) for i in range(1,10)]))
                     ), axis=1)

In [431]:
G2 = add_link_ratios_from_raw_data(N1)

In [432]:
%timeit G2 = add_link_ratios_from_raw_data(N1)

44.8 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
G2.xs(sfm, level=0).filter(regex='Paid').head(10)

In [ ]:
G2.loc[sfm, 'PaidLink'].head(10)

In [ ]:
# just the complete triangles 
comp = G2.loc[G2.groupby(['GRName', 'Line']).apply(lambda x : x.isna().sum().sum()) == 180, :]

In [ ]:
G2.shape, comp.shape

In [433]:
def mask(n, size, kind):
    """ 
    mask for avg last n in a size x size triangle 
    """
    nyrs = size - 1
    if kind=='loss_den':
        ans = np.array([[1 if i + j < nyrs and i + j >= nyrs - n else 0 for i in range(size)] for j in range(size)])
    elif kind=='loss_num':
        ans = np.array([[1 if i > 0 and i + j < size and i + j >= size - n else 0 for i in range(size)] for j in range(size)])
    else:
        ans = np.array([[1 if i + j < nyrs and i + j >= nyrs - n else 0 for i in range(nyrs)] for j in range(size)])
    return ans

def make_links(x, avg_tuple=(3, 5, 10)):
    '''
    Compute paid and incurred average link ratios, weight and straight, 3, 5 and all year averages (2x2x3=12 sets)
    '''
    return pd.DataFrame({ \
        **{ ('Inc', f'str {i}') : np.nansum(x.loc[:, 'CaseIncLink'].values * mask(i, 10, 'link'), 0) / np.nansum( mask(i, 10, 'link'), 0) for i in avg_tuple}, \
        **{ ('Pd', f'str {i}') :  np.nansum(x.loc[:, 'PaidLink'].values * mask(i, 10, 'link'), 0) /  np.nansum( mask(i, 10, 'link'), 0) for i in avg_tuple}, \
        **{ ('Inc', f'wtd {i}') : np.nansum((x.loc[:, 'CaseIncLoss'].values * mask(i, 10, 'loss_num')), 0)[1:] / \
                                  np.nansum((x.loc[:, 'CaseIncLoss'].values * mask(i, 10, 'loss_den')), 0)[:-1] for i in avg_tuple}, \
        **{ ('Pd', f'wtd {i}') :  np.nansum((x.loc[:, 'PaidLoss'].values * mask(i, 10, 'loss_num')), 0)[1:] / \
                                  np.nansum((x.loc[:, 'PaidLoss'].values * mask(i, 10, 'loss_den')), 0)[:-1] for i in avg_tuple}, \
        }, \
        index=pd.Index(range(1,10), name='Lag')).T

# def make_links2(x, avg_tuple=(3, 5, 10)):
#     '''
#     Compute paid and incurred average link ratios, weight and straight, 3, 5 and all year averages (2x2x3=12 sets)
#     Use masked arrays and dictionary list comprehensions 
#     SLOWER but probably correct for incomplete triangles.... 
#     '''
#     return pd.DataFrame({ \
#         **{ (j, f'str {i}') : ma.masked_array(x.loc[:, j], mask(i, 10, 'link')).mean(0) \
#            for i in avg_tuple for j in ['CaseIncLink', 'PaidLink']}, \
#         **{ (j, f'wtd {i}') : ma.masked_array(x.loc[:, k], mask(i, 10, 'loss_num')).sum(0)[1:] / \
#            ma.masked_array(x.loc[:, k], mask2(i, 10, 'loss_den')).sum(0)[:-1] \
#            for i in avg_tuple for j, k in [('Inc', 'CaseIncLoss'), ('Pd', 'PaidLoss')]}, \
#         }, \
#         index=pd.Index(range(1,10), name='Lag')).T

In [434]:
links = comp.groupby(level=['GRName', 'Line']).apply(make_links)
links.index.names = ['GRName', 'Line', 'Kind', 'Method']

In [436]:
def maskex(n, size, kind, tiles):
    """ 
    mask for avg last n in a size x size triangle 
    """
    nyrs = size - 1
    if kind=='loss_den':
        ans = np.array([[1 if i + j < nyrs and i + j >= nyrs - n else 0 for i in range(size)] for j in range(size)])
    elif kind=='loss_num':
        ans = np.array([[1 if i > 0 and i + j < size and i + j >= size - n else 0 for i in range(size)] for j in range(size)])
    else:
        ans = np.array([[1 if i + j < nyrs and i + j >= nyrs - n else 0 for i in range(nyrs)] for j in range(size)])
    return np.tile(ans, (tiles, 2))

def mask_count(n, size):
    n = min(n, size-1)
    return np.tile(np.array([n]*(size-n-1) + list(range(n,0,-1))), 2)
complete=comp
bril = pd.concat([(complete.filter(regex='Loss', axis=1) * maskex(i, 10, 'loss_num', 400)).iloc[:, pd.np.r_[1:10, 11:20]].groupby(level=[0,1]).sum().values / \
           (complete.filter(regex='Loss', axis=1) * maskex(i, 10, 'loss_den', 400)).iloc[:, pd.np.r_[0:9, 10:19]].groupby(level=[0,1]).sum() for i in [3, 5, 10]]+
           [(complete.filter(regex='Link', axis=1) * maskex(i, 10, 'link', 400)).groupby(level=[0,1]).sum() / mask_count(i, 10) for i in [3, 5, 10]],
                    axis=1,
                 keys=[(wt, i) for wt in ['Wtd', 'Str'] for i in [3, 5, 10]] ) 
bril.columns.names= ['Method', 'Nyrs', 'LossType', 'DY']
bril = bril.stack(level=(0,1,2))

In [437]:
bril.loc[sfm].head(12).sort_index(level=[0,1,1,3])

DY                                        1         2         3         4  \
Line      Method Nyrs LossType                                              
Comm Auto Str    3    CaseIncLink  1.293552  1.113861  1.060338  1.026891   
                 5    CaseIncLink  1.313512  1.121383  1.064900  1.026766   
                 10   CaseIncLink  1.339910  1.122061  1.065689  1.027940   
                 3    PaidLink     1.826069  1.263130  1.128121  1.062332   
                 5    PaidLink     1.890888  1.276082  1.136528  1.062149   
                 10   PaidLink     1.929178  1.286610  1.140512  1.066676   
          Wtd    3    CaseIncLoss  1.293524  1.113678  1.060463  1.026928   
                 5    CaseIncLoss  1.312186  1.120616  1.064592  1.026813   
                 10   CaseIncLoss  1.336434  1.121964  1.065358  1.027861   
                 3    PaidLoss     1.826329  1.262811  1.127585  1.061927   
                 5    PaidLoss     1.884683  1.274334  1.135774  1.061906   
                 10   PaidLoss     1.920529  1.284353  1.139525  1.065953   

DY                                        5         6         7         8  \
Line      Method Nyrs LossType                                              
Comm Auto Str    3    CaseIncLink  1.018002  1.010766  1.005025  1.004704   
                 5    CaseIncLink  1.016194  1.009682  1.005025  1.004704   
                 10   CaseIncLink  1.016194  1.009682  1.005025  1.004704   
                 3    PaidLink     1.039262  1.016685  1.008713  1.007015   
                 5    PaidLink     1.037350  1.016063  1.008713  1.007015   
                 10   PaidLink     1.037350  1.016063  1.008713  1.007015   
          Wtd    3    CaseIncLoss  1.017973  1.010706  1.004824  1.004742   
                 5    CaseIncLoss  1.016305  1.009717  1.004824  1.004742   
                 10   CaseIncLoss  1.016305  1.009717  1.004824  1.004742   
                 3    PaidLoss     1.039367  1.016560  1.008650  1.007079   
                 5    PaidLoss     1.037541  1.016012  1.008650  1.007079   
                 10   PaidLoss     1.037541  1.016012  1.008650  1.007079   

DY                                        9  
Line      Method Nyrs LossType               
Comm Auto Str    3    CaseIncLink  1.011117  
                 5    CaseIncLink  1.011117  
                 10   CaseIncLink  1.011117  
                 3    PaidLink     1.015636  
                 5    PaidLink     1.015636  
                 10   PaidLink     1.015636  
          Wtd    3    CaseIncLoss  1.011117  
                 5    CaseIncLoss  1.011117  
                 10   CaseIncLoss  1.011117  
                 3    PaidLoss     1.015636  
                 5    PaidLoss     1.015636  
                 10   PaidLoss     1.015636

In [438]:
links.loc[sfm].head(12)

Lag                           1         2         3         4         5  \
Line      Kind Method                                                     
Comm Auto Inc  str 3   1.293552  1.113861  1.060338  1.026891  1.018002   
               str 5   1.313512  1.121383  1.064900  1.026766  1.016194   
               str 10  1.339910  1.122061  1.065689  1.027940  1.016194   
          Pd   str 3   1.826069  1.263130  1.128121  1.062332  1.039262   
               str 5   1.890888  1.276082  1.136528  1.062149  1.037350   
               str 10  1.929178  1.286610  1.140512  1.066676  1.037350   
          Inc  wtd 3   1.293524  1.113678  1.060463  1.026928  1.017973   
               wtd 5   1.312186  1.120616  1.064592  1.026813  1.016305   
               wtd 10  1.336434  1.121964  1.065358  1.027861  1.016305   
          Pd   wtd 3   1.826329  1.262811  1.127585  1.061927  1.039367   
               wtd 5   1.884683  1.274334  1.135774  1.061906  1.037541   
               wtd 10  1.920529  1.284353  1.139525  1.065953  1.037541   

Lag                           6         7         8         9  
Line      Kind Method                                          
Comm Auto Inc  str 3   1.010766  1.005025  1.004704  1.011117  
               str 5   1.009682  1.005025  1.004704  1.011117  
               str 10  1.009682  1.005025  1.004704  1.011117  
          Pd   str 3   1.016685  1.008713  1.007015  1.015636  
               str 5   1.016063  1.008713  1.007015  1.015636  
               str 10  1.016063  1.008713  1.007015  1.015636  
          Inc  wtd 3   1.010706  1.004824  1.004742  1.011117  
               wtd 5   1.009717  1.004824  1.004742  1.011117  
               wtd 10  1.009717  1.004824  1.004742  1.011117  
          Pd   wtd 3   1.016560  1.008650  1.007079  1.015636  
               wtd 5   1.016012  1.008650  1.007079  1.015636  
               wtd 10  1.016012  1.008650  1.007079  1.015636

In [440]:
%%timeit
bril = pd.concat([(complete.filter(regex='Loss', axis=1) * maskex(i, 10, 'loss_num', 400)).iloc[:, pd.np.r_[1:10, 11:20]].groupby(level=[0,1]).sum().values / \
           (complete.filter(regex='Loss', axis=1) * maskex(i, 10, 'loss_den', 400)).iloc[:, pd.np.r_[0:9, 10:19]].groupby(level=[0,1]).sum() for i in [3, 5, 10]]+
           [(complete.filter(regex='Link', axis=1) * maskex(i, 10, 'link', 400)).groupby(level=[0,1]).sum() / mask_count(i, 10) for i in [3, 5, 10]],
                    axis=1,
                 keys=[(wt, i) for wt in ['Wtd', 'Str'] for i in [3, 5, 10]] ) 
bril.columns.names= ['Method', 'Nyrs', 'LossType', 'DY']
bril = bril.stack(level=(0,1,2))

305 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [441]:
%%timeit
links0 = complete.groupby(level=['GRName', 'Line']).apply(make_links)
links0.index.names = ['GRName', 'Line', 'Kind', 'Method']
links0.head(20)

3.54 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# sfm ppa results
links.loc[[sfm]].iloc[24:36, :]

#  Bootstrapping 

In [ ]:
def pd_inc_plot(df, co_name='', line_name='', bins=201, dd=True, ax=None, legend=False):
    '''
    bootstrap from paid and incurred and create product distribution 
    input is result of running
    
        links = comp.groupby(level=['GRName', 'Line']).apply(make_links)
        links.index.names = ['GRName', 'Line', 'Kind', 'Method']
    
    i.e. df has index GRName, Line, AY and col groups for Paid, CaseInc loss and links  and lag 
    '''

    def shorten(s):
        '''
        name shortening function for labels 
        '''
        if len(s) < 12:
            return s
        else:
            re.sub
            s = re.sub(' (Co|Ins|Grp|Exchange|Of|Inc|of)', '', s)
            s = s.replace('Agricultural', 'Ag').replace('Exchange', 'Ex'). replace('Associated', 'Assoc')
        if len(s) > 12:
            s = ' '.join([i[:4] for i in s.split(' ')][:3])
        return s
    # allows use with groupby
    if co_name == '':
        co_name, line_name, _ = df.index[0]
   
    yrs = list(df.index.get_level_values('AY').unique())
    nyrs = yrs[-1] - yrs[0]
    
    # piece of interest
    bit = df.xs((co_name, line_name), level=('GRName', 'Line'))
    
    if len(bit) < 10:
        return
    
    # make kronecker products 
    # pull off most recent year losses 
    kpi = np.array(bit.loc[yrs[-1], ('CaseIncLoss', 1)])
    kpp = np.array(bit.loc[yrs[-1], ('PaidLoss', 1)])
    
    # and complete with link ratios 
    for i in range(0, nyrs):
        kpp = np.kron(kpp, bit.loc[yrs[0]:yrs[0]+i, ('PaidLink', nyrs - i)])
        kpi = np.kron(kpi, bit.loc[yrs[0]:yrs[0]+i, ('CaseIncLink', nyrs - i)])

    ult = pd.DataFrame( {'inc' : kpi, 'pd' : kpp})
    # stats 
    d = ult.describe().iloc[1:, :]
    if dd:
        display(d)
    
    if ax is None:
        f = plt.figure()
        a = f.gca()
    else:
        a = next(ax)
    
    bp = np.linspace(d.loc['min', :].min(), d.loc['max', :].max(), bins)
    mnn = d.loc['mean', :].min()
    mnx = d.loc['mean', :].max()
    sd = d.loc['std', : ].max()
    bp = np.linspace(max(0, mnn - 4*sd), mnx + 4*sd, bins)
    npd,  _, _ = a.hist(kpp, bins=bp, color='b', alpha=0.5, label='Paid')
    ninc, _, _ = a.hist(kpi, bins=bp, color='r', alpha=0.5, label='Incurred')
    bay = ninc*npd / sum(ninc*npd) * sum(npd)
    xs = (bp[1:]+bp[0:-1])/2
    a.plot(xs, bay, '-g', label='Posterior')
    if legend:
        a.legend(frameon=False)
    a.set(title='{:}/{:}\nMLE={:,.1f}, CV(I/Pd)={:.3f}/{:.3f}'.format(shorten(co_name), line_name, xs[bay.argmax()]/1e3, 
                                                                *(d.loc['std']/d.loc['mean']) ))
    return ult

In [ ]:
lines = list(N1.Line.unique())
lines

In [ ]:
f, ax = plt.subplots(2, 3, figsize=(12,8))
ax = iter(ax.flatten())
for l in lines:
    ult = pd_inc_plot(comp, sfm, l, dd=False, ax=ax, legend=(l==lines[0]))
# tidy up 
for a in ax:
    f.delaxes(a)
plt.tight_layout()

In [ ]:
def plot_all(df, line='', co='', threshold=250000):
    '''
    all lines for given co or all cos for given line 
    '''
    if line=='' and co=='':
        return 
    
    if line != '':
        bit = df.query(f' Line=="{line}" ')        
        ncos = len(bit) / 10 
        nr = int(ncos/6)
        if nr < ncos/6: nr += 1
        f, ax = plt.subplots(nr, 6, figsize=(18, 2.4*nr))
        ax = iter(ax.flatten())
        
    elif co != '':
        bit = df.query(f' GRName=="{co}" ')
        f, ax = plt.subplots(2, 3, figsize=(12,6))
        ax = iter(ax.flatten())
    
    g = bit.groupby(['GRName', 'Line'])

    l = True
    for k, v in g.groups.items():
        grp = bit.loc[v]
        if grp.CaseIncLoss.sum().sum() > threshold:
            ult = pd_inc_plot(grp, dd=False, ax=ax, legend=l)
            l = False
        
    # tidy up 
    for a in ax:
        f.delaxes(a)
    plt.tight_layout()

In [ ]:
[i for i in comp.index.get_level_values('GRName').unique() if i[:5] == 'Canal']

In [ ]:
plot_all(comp, 'PP Auto', 100000)

In [ ]:
plot_all(comp, 'Work Comp', 1000000)

# Data For SciKit-Learn Intro --> For this afternoon's session

In [ ]:
# Read in the CAS data
data_url = 'https://www.casact.org/research/reserve_data'
lobs = ['medmal','ppauto','wkcomp', 'othliab', 'comauto', 'prodliab']
data = pd.DataFrame()
data = []
columns = ['GRCODE','GRNAME','AccidentYear','DevelopmentYear','DevelopmentLag'
           ,'IncurLoss', 'CumPaidLoss','BulkLoss','EarnedPremDIR'
           ,'EarnedPremCeded','EarnedPremNet', 'Single','PostedReserve97']
for lob in lobs:
    file_url = f'{data_url}/{lob}_pos.csv'
    subset = pd.read_csv(file_url, names=columns, skiprows=1)
    subset['LOB'] = lob
    data.append(subset)
data1 = pd.concat(data)
data = data1.query(" DevelopmentYear <= 1997 ").reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
def make_triangles(data, nlarge=20):
    '''
    make ldf triangles from CAS data for largest companies
    '''
    aggregates2 = data.query(' DevelopmentYear ==  1997 ').groupby(['LOB','GRNAME'])['IncurLoss'].sum() 
    top_by_lob = aggregates2.groupby(level='LOB').apply(lambda x : x.nlargest(nlarge).reset_index(level=0, drop=True))
    
    data_alt2 = data.merge(top_by_lob.to_frame(), how='left', left_on=['LOB','GRNAME'], right_index=True)
    data_alt2.loc[data_alt2.loc[:,'IncurLoss_y'].isna(), 'GRNAME'] = 'Other'
    
    # create triangles 
    triangles = pd.pivot_table(data_alt2, index=['GRNAME','LOB','AccidentYear'],
                           columns='DevelopmentLag', values='CumPaidLoss')
    
    # Determine LDF Weights ORIG
    w = pd.DataFrame(np.array([[1 if i+j<9 else 0 for i in range(9)] for j in range(10)]))
    weight = np.tile(w, (int(triangles.shape[0]/10), 1))
    columns = [f'{triangles.columns[num]}-{triangles.columns[num+1]}'
               for num, item in enumerate(triangles.columns[:-1])]

    # Volume-weighted numerator and demoninator mask for denom only; values on num because want index from num 
    ldf = (triangles.iloc[:,1:].groupby(level=['GRNAME','LOB']).sum().values / \
           (weight*triangles.iloc[:,:-1]).groupby(level=['GRNAME','LOB']).sum()).fillna(1.0) 
    return ldf

In [ ]:
ldfs = make_triangles(data, 20)
plot_data = pd.DataFrame(ldfs.stack(), columns=['avg_link']).reset_index()
plot_data.head()

In [ ]:
sns.relplot(data=plot_data, kind='line', x='DevelopmentLag', y='avg_link')

In [ ]:
plot_data.head()

In [ ]:
plot_data['trans'] = np.log(plot_data.avg_link - 1)

In [ ]:
a = sns.relplot(data=plot_data.query('DevelopmentLag >= 1'), kind='line', x='DevelopmentLag', y='trans', hue='GRNAME', col='LOB', col_wrap=3, legend=False)
# for ax in a.axes.flatten():
#     ax.set(ylim=[0.9,1.5])